# Weather Station

while read line; do
    echo $line;
    zzz=$(( $RANDOM % 3 ))
    sleep 0.${zzz}
done < $1 | nc -lk $2


OKOK


In [1]:
from pyspark.streaming import StreamingContext

# The "1" here is the number of seconds between microbatches:
ssc = StreamingContext(sc, 0.5)
# Required to be able to do state updates:
ssc.checkpoint("checkpoint")
#ssc.checkpoint("file:///home/mchen81/Project3/spark-tmp")

In [2]:
# Updates our distribution
def update_temp(new_values, old_values):
    return sum(new_values) + (old_values or 0)

In [3]:
import geohash
import datetime

#  surface temperature, pressure, humidity, precipitation, visibility, and wind speed
def parseLine(line):    
    variables = line.split("\t")
    try:
        milliseconds = int(variables[0])
        dt = datetime.datetime.fromtimestamp(milliseconds/1000.0)
        lat = float(variables[1])
        lon = float(variables[2])
        pressure = float(variables[6])
        humidity = float(variables[8])
        temperature = float(variables[10]) - 273.15 #in celcius
        precipitation = float(variables[13])
        visibility = float(variables[15])
        # wind_speed = float(variables[17] == 'null' if 0 else variables[17])
        gh = geohash.encode(lat, lon)
        return (gh[0:3], dt.month, pressure, humidity, temperature, precipitation, visibility)

    except:
        return ('', 0)

In [4]:
# For a new value newValue, compute the new count, new mean, the new M2.
# mean accumulates the mean of the entire dataset
# M2 aggregates the squared distance from the mean
# count aggregates the number of samples seen so far

def update_mean(newValues, existingAggregate):    
    (count, mean, M2) = (existingAggregate or (0, 0, 0))
    for newValue in newValues:
        count += 1
        delta = newValue - mean
        mean += delta / count
        delta2 = newValue - mean
        M2 += delta * delta2
    return (count, mean, M2)

# Retrieve the mean, variance and sample variance from an aggregate
def finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    if count < 2:
        return float("nan")
    else:
        (mean, variance, sampleVariance) = (mean, M2 / count, M2 / (count - 1))
        return [mean, variance, sampleVariance]

In [5]:
BOSTON = 'drt'
SD = '9mu'
SF = '9q8'
RICHMOND = '9q9'
LA = '9q5'
def check_location(gh):
    return  gh == BOSTON or \
            gh == SD or \
            gh == SF or \
            gh == RICHMOND or \
            gh == LA    

In [6]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython import display

# Plots the distribution. This executes on the driver!
def plot_distribution(rdd):
    
    display.clear_output(wait=True)
    df = pd.DataFrame(rdd, columns=['location', 'mean', 'variance', 'sampleVariance'])
    df.sort_values('location', ascending=True) \
        .plot(kind='bar', x='location', y=['mean', 'variance', 'sampleVariance'])
    #plt.ylim(-10, 20)
    display.display(plt.gcf())
    #plt.clf()
    plt.close()
    #plt.show()


In [7]:
rdd_tmp_file = "weather-station-result/file.txt"
f = open(rdd_tmp_file, "w")
f.write("")
f.close()

def write_to_file(rdd):
    f = open(rdd_tmp_file, "a")
    f.write("============================\n")
    f.write(str(rdd))
    f.write("\n")
    f.close()

    

FileNotFoundError: [Errno 2] No such file or directory: 'weather-station-result/file.txt'

In [8]:
lines = ssc.socketTextStream("orion03", 12889)
weather_info = lines.map(lambda line: parseLine(line))
filtered_location = weather_info.filter(lambda t: check_location(t[0]))
location_pressure = filtered_location.map(lambda t: (t[0], t[4])) # geo-hash -> pressure
distrib = location_pressure.updateStateByKey(update_mean)
finalized = distrib.map(lambda dis: [dis[0]] + finalize(dis[1]))
#finalized.foreachRDD(lambda rdd: write_to_file(rdd.collect()))
# finalized.foreachRDD(lambda rdd: plot_distribution(rdd.collect()))
finalized.pprint()
#finalized.foreachRDD(lambda rdd: rdd.foreach(ppp))
# display.display(plt.gcf())
ssc.start()
#ssc.awaitTermination()

-------------------------------------------
Time: 2020-11-29 06:34:36
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:36.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:37
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:37.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:38
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:38.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:39
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:34:39.500000
-------------------------------------------

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2020-11-29 06:35:11
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:11.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:12
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:12.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:13
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:13.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:14
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:14.500000
-------------------------------------------

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2020-11-29 06:35:46
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:46.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:47
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:47.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:48
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:48.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:49
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:35:49.500000
-------------------------------------------

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2020-11-29 06:36:21
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:21.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:22
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:22.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:23
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:23.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:24
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:24.500000
-------------------------------------------

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2020-11-29 06:36:56
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:56.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:57
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:57.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:58
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:58.500000
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:59
-------------------------------------------

-------------------------------------------
Time: 2020-11-29 06:36:59.500000
-------------------------------------------

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2020-11-29 06:37:10
-------------------------------------------
['drt', -1.071703814541606, 21.925694847340147, 21.948823217432277]
['9q8', 13.798713167898622, 3.8513065144881695, 3.855377663023569]
['9q5', 5.613253739495807, 84.36989899595308, 84.45861602959761]
['9mu', 11.160776147623874, 58.170670032883656, 58.22954722927321]
['9q9', 4.3693928616025115, 51.35877615596493, 51.412274881127395]

-------------------------------------------
Time: 2020-11-29 06:37:10.500000
-------------------------------------------
['drt', -1.071703814541606, 21.925694847340147, 21.948823217432277]
['9q8', 13.798713167898622, 3.8513065144881695, 3.855377663023569]
['9q5', 5.613253739495807, 84.36989899595308, 84.45861602959761]
['9mu', 11.160776147623874, 58.170670032883656, 58.22954722927321]
['9q9', 4.3693928616025115, 51.35877615596493, 51.412274881127395]



In [64]:
ssc.stop(stopSparkContext=False)


-------------------------------------------
Time: 2020-11-28 20:39:37
-------------------------------------------

-------------------------------------------
Time: 2020-11-28 20:39:37.500000
-------------------------------------------

